###### Imports and Settings

In [24]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)

# This notebook outlines the download and formatting process for data from the Tennessee Bureau of Investigation.  

Go to this page: https://crimeinsight.tbi.tn.gov/public/View/dispview.aspx?ReportId=20, "Crime Rates by Jurisdiction" fom this page: https://crimeinsight.tbi.tn.gov/public/Browse/browsetables.aspx
We are looking for Crime Rates by county and the State of Tennessee here. Select "Jurisdiction by Geography" on the left-side of the screen, where you are then able to manipulate geographies by level. Select the State, and then deselect the regions, select counties in the GNRC operating area (most in Middle, some in West), and under those counties leave the jurisdictions. Click "show report" in the top right-side of the screen, and then download a multi-dimensional csv file. Delete the data headers, rename Jurisdiction by Geography to "NAME", make sure that the numbers are not formatted with commas, add a county column to tag the juridictions with a county, leaving the column null for the state and counties, and set the column headers correctly before saving the file as "TBRCrimeRatesbyJurisdiction_IncidentDate(YYYY).csv.  

Next, From the homepage select "Crime and Clearance by Offense Type", deselect clearance in the left-side selector. Select "Jurisdiction by Geography" and similarly select the State and all Counties and jurisdictions. Download this, format, transpose data so NAME is on the left and you can create a similar county column as before, and save as 

*Note*: the jurisdictions don't necessarily repond to a place level plan in all cases, but they may and you can toy around here to see if they match the plan or document you're generating data for!


In [25]:
data = pd.read_csv('../../Data Downloads/TBI_CrimeRatesbyJurisdiction_IncidentDate2021.csv')
data.head()

,NAME,County,"Crime Rate (per 1,000)",Estimated Population,Number of Crimes
0,Tennessee,NaN,72.64,6975218.0,506713.0
1,Cheatham,NaN,48.98,41483.0,2032.0
2,Cheatham County Sheriff's Office,Cheatham,NaN,NaN,1262.0
3,Ashland City Police Department,Cheatham,125.55,4763.0,598.0
4,Kingston Springs Police Department,Cheatham,26.18,2712.0,71.0


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NAME                    138 non-null    object 
 1   County                  123 non-null    object 
 2   Crime Rate (per 1,000)  70 non-null     float64
 3   Estimated Population    80 non-null     float64
 4   Number of Crimes        106 non-null    float64
dtypes: float64(3), object(2)
memory usage: 5.5+ KB


In [27]:
#data['NAME'] = data['NAME'] + ' County'

In [28]:
data.to_feather('../../Outputs/TBI_CrimeRate_2021')

In [29]:
data = pd.read_csv('../../Data Downloads/TBI_CrimeTypebyJurisdiction_IncidentDate2021.csv')
data.head()

,NAME,County,Murder,Negligent Manslaughter,Negligent Vehicular Manslaughter,Kidnapping/Abduction,Forcible Rape,Forcible Sodomy,Sexual Assault W/Object,Forcible Fondling,Incest,Statutory Rape,Aggravated Assault,Simple Assault,Intimidation,Stalking,Commercial Sex Acts,Involuntary Servitude,Arson,Bribery,Burglary,Counterfeiting/Forgery,Destruction/Damage/Vandalism,Embezzlement,Extortion/Blackmail,Fraud - False Pretenses,Fraud - Credit Card/ATM,Fraud - Impersonation,Fraud - Welfare,Fraud - Wire,Fraud - Identity Theft,Fraud - Computer Hacking/Invasion,Robbery,Theft - Pocket-picking,Theft - Purse Snatching,Theft - Shoplifting,Theft From Building,Theft From Coin Machine,Theft From Motor Vehicle,Theft of Motor Vehicle Parts,Theft - All Other Larceny,Motor Vehicle Theft,Stolen Property Offenses,Animal Cruelty,Drug/Narcotic Violations,Drug/Narcotic Equipment Violations,Gambling - Betting/Wagering,Gambling - Operating/Promoting,Gambling - Equipment Violations,Gambling - Sports Tampering,Pornography/Obscene Material,Prostitution,Prostitution Assisting/Promoting,Purchasing Prostitution,Weapon Law Violations
0,Tennessee,NaN,704.0,44.0,21.0,2206.0,2373.0,457.0,289.0,2744.0,27.0,367.0,38501.0,74668.0,25087.0,1847.0,118.0,2.0,955.0,26.0,23027.0,4795.0,46355.0,1577.0,630.0,12855.0,7118.0,3994.0,184.0,1629.0,441.0,166.0,4961.0,476.0,149.0,27335.0,16324.0,207.0,29676.0,13216.0,27883.0,21597.0,2806.0,588.0,55724.0,35045.0,9.0,14.0,9.0,NaN,1048.0,246.0,28.0,135.0,16030.0
1,Cheatham,NaN,2.0,NaN,NaN,8.0,7.0,NaN,NaN,9.0,NaN,1.0,86.0,268.0,79.0,7.0,NaN,NaN,1.0,NaN,58.0,35.0,156.0,7.0,NaN,56.0,17.0,33.0,NaN,22.0,NaN,NaN,3.0,NaN,NaN,59.0,75.0,1.0,46.0,54.0,96.0,59.0,18.0,2.0,431.0,280.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,55.0
2,Cheatham County Sheriff's Office,Cheatham,2.0,NaN,NaN,7.0,6.0,NaN,NaN,6.0,NaN,NaN,68.0,210.0,58.0,5.0,NaN,NaN,1.0,NaN,38.0,15.0,110.0,1.0,NaN,12.0,11.0,28.0,NaN,22.0,NaN,NaN,2.0,NaN,NaN,8.0,61.0,NaN,31.0,41.0,54.0,42.0,15.0,2.0,234.0,137.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,34.0
3,Ashland City Police Department,Cheatham,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0,12.0,45.0,17.0,NaN,NaN,NaN,NaN,NaN,19.0,16.0,30.0,5.0,NaN,40.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,7.0,1.0,2.0,6.0,32.0,9.0,2.0,NaN,165.0,118.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
4,Kingston Springs Police Department,Cheatham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,7.0,5.0,1.0,NaN,21.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [30]:
data = data.set_index(['NAME', 'County'])

In [31]:
data.fillna(0, inplace = True)

In [32]:
data.head()

,,Murder,Negligent Manslaughter,Negligent Vehicular Manslaughter,Kidnapping/Abduction,Forcible Rape,Forcible Sodomy,Sexual Assault W/Object,Forcible Fondling,Incest,Statutory Rape,Aggravated Assault,Simple Assault,Intimidation,Stalking,Commercial Sex Acts,Involuntary Servitude,Arson,Bribery,Burglary,Counterfeiting/Forgery,Destruction/Damage/Vandalism,Embezzlement,Extortion/Blackmail,Fraud - False Pretenses,Fraud - Credit Card/ATM,Fraud - Impersonation,Fraud - Welfare,Fraud - Wire,Fraud - Identity Theft,Fraud - Computer Hacking/Invasion,Robbery,Theft - Pocket-picking,Theft - Purse Snatching,Theft - Shoplifting,Theft From Building,Theft From Coin Machine,Theft From Motor Vehicle,Theft of Motor Vehicle Parts,Theft - All Other Larceny,Motor Vehicle Theft,Stolen Property Offenses,Animal Cruelty,Drug/Narcotic Violations,Drug/Narcotic Equipment Violations,Gambling - Betting/Wagering,Gambling - Operating/Promoting,Gambling - Equipment Violations,Gambling - Sports Tampering,Pornography/Obscene Material,Prostitution,Prostitution Assisting/Promoting,Purchasing Prostitution,Weapon Law Violations
NAME,County,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Tennessee,NaN,704.0,44.0,21.0,2206.0,2373.0,457.0,289.0,2744.0,27.0,367.0,38501.0,74668.0,25087.0,1847.0,118.0,2.0,955.0,26.0,23027.0,4795.0,46355.0,1577.0,630.0,12855.0,7118.0,3994.0,184.0,1629.0,441.0,166.0,4961.0,476.0,149.0,27335.0,16324.0,207.0,29676.0,13216.0,27883.0,21597.0,2806.0,588.0,55724.0,35045.0,9.0,14.0,9.0,0.0,1048.0,246.0,28.0,135.0,16030.0
Cheatham,NaN,2.0,0.0,0.0,8.0,7.0,0.0,0.0,9.0,0.0,1.0,86.0,268.0,79.0,7.0,0.0,0.0,1.0,0.0,58.0,35.0,156.0,7.0,0.0,56.0,17.0,33.0,0.0,22.0,0.0,0.0,3.0,0.0,0.0,59.0,75.0,1.0,46.0,54.0,96.0,59.0,18.0,2.0,431.0,280.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,55.0
Cheatham County Sheriff's Office,Cheatham,2.0,0.0,0.0,7.0,6.0,0.0,0.0,6.0,0.0,0.0,68.0,210.0,58.0,5.0,0.0,0.0,1.0,0.0,38.0,15.0,110.0,1.0,0.0,12.0,11.0,28.0,0.0,22.0,0.0,0.0,2.0,0.0,0.0,8.0,61.0,0.0,31.0,41.0,54.0,42.0,15.0,2.0,234.0,137.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,34.0
Ashland City Police Department,Cheatham,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,12.0,45.0,17.0,0.0,0.0,0.0,0.0,0.0,19.0,16.0,30.0,5.0,0.0,40.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,7.0,1.0,2.0,6.0,32.0,9.0,2.0,0.0,165.0,118.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
Kingston Springs Police Department,Cheatham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,7.0,5.0,1.0,0.0,21.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [33]:
thelist = [data['Negligent Manslaughter'], data['Negligent Vehicular Manslaughter']]
data['Manslaughter'] = sum(thelist)
thelist = [data['Forcible Rape'], data['Forcible Sodomy'], data['Sexual Assault W/Object'], data['Forcible Fondling'], data['Incest'], data['Statutory Rape'], 
          data['Aggravated Assault'], data['Simple Assault']]
data['Assault & Violent Sex Crime'] = sum(thelist)
thelist = [data['Intimidation'], data['Stalking']]
data['Intimidation & Stalking'] = sum(thelist)
thelist = [data['Arson'], data['Destruction/Damage/Vandalism']]
data['Arson, Destruction & Vandalism'] = sum(thelist)
thelist = [data['Fraud - False Pretenses'], data['Fraud - Credit Card/ATM'], data['Fraud - Impersonation'], data['Fraud - Welfare'], data['Fraud - Wire'], 
          data['Fraud - Identity Theft'], data['Fraud - Computer Hacking/Invasion']]
data['Fraud'] = sum(thelist)
thelist = [data['Theft - Pocket-picking'], data['Theft - Purse Snatching'], data['Theft - Shoplifting'], data['Theft From Building'], 
           data['Theft From Coin Machine'], data['Theft From Motor Vehicle'], data['Theft of Motor Vehicle Parts'], data['Theft - All Other Larceny'], 
           data['Motor Vehicle Theft'], data['Stolen Property Offenses']]
data['Theft'] = sum(thelist)
thelist = [data['Drug/Narcotic Violations'], data['Drug/Narcotic Equipment Violations']]
data['Drugs & Narcotics Violations'] = sum(thelist)
thelist = [data['Bribery'], data['Counterfeiting/Forgery'], data['Embezzlement'], data['Extortion/Blackmail'],
           data['Gambling - Betting/Wagering'], data['Gambling - Operating/Promoting'], data['Gambling - Equipment Violations'], 
           data['Gambling - Sports Tampering']]
data['Bribery, Forgery, Blackmail, Embezzlement & Gambling'] = sum(thelist)
thelist = [data['Prostitution'], data['Prostitution Assisting/Promoting'], data['Purchasing Prostitution'], 
           data['Commercial Sex Acts'], data['Pornography/Obscene Material']]
data['Prostitution, Commercial Sex Acts & Pornography, Aiding & Purchasing Prostitution'] = sum(thelist)
cols = ['Negligent Manslaughter','Negligent Vehicular Manslaughter','Forcible Rape','Forcible Sodomy','Sexual Assault W/Object','Forcible Fondling','Incest',
        'Statutory Rape','Aggravated Assault','Simple Assault','Intimidation','Stalking','Arson','Destruction/Damage/Vandalism','Bribery','Counterfeiting/Forgery',
        'Embezzlement','Extortion/Blackmail','Fraud - False Pretenses','Fraud - Credit Card/ATM','Fraud - Impersonation','Fraud - Welfare','Fraud - Wire',
        'Fraud - Identity Theft','Fraud - Computer Hacking/Invasion', 'Theft - Pocket-picking','Theft - Purse Snatching','Theft - Shoplifting','Theft From Building',
        'Theft From Coin Machine','Theft From Motor Vehicle','Theft of Motor Vehicle Parts','Theft - All Other Larceny','Motor Vehicle Theft',
        'Commercial Sex Acts', 'Pornography/Obscene Material',
        'Stolen Property Offenses','Drug/Narcotic Violations','Drug/Narcotic Equipment Violations','Gambling - Betting/Wagering','Gambling - Operating/Promoting',
        'Gambling - Equipment Violations','Gambling - Sports Tampering','Prostitution','Prostitution Assisting/Promoting','Purchasing Prostitution', 
        'Involuntary Servitude']# there is just hardly any of this I am dropping it for this analysis
data = data.drop(columns = cols)

In [34]:
data = data.reset_index()

In [35]:
data.head()

,NAME,County,Murder,Kidnapping/Abduction,Burglary,Robbery,Animal Cruelty,Weapon Law Violations,Manslaughter,Assault & Violent Sex Crime,Intimidation & Stalking,"Arson, Destruction & Vandalism",Fraud,Theft,Drugs & Narcotics Violations,"Bribery, Forgery, Blackmail, Embezzlement & Gambling","Prostitution, Commercial Sex Acts & Pornography, Aiding & Purchasing Prostitution"
0,Tennessee,NaN,704.0,2206.0,23027.0,4961.0,588.0,16030.0,65.0,119426.0,26934.0,47310.0,26387.0,139669.0,90769.0,7060.0,1575.0
1,Cheatham,NaN,2.0,8.0,58.0,3.0,2.0,55.0,0.0,371.0,86.0,157.0,128.0,408.0,711.0,42.0,1.0
2,Cheatham County Sheriff's Office,Cheatham,2.0,7.0,38.0,2.0,2.0,34.0,0.0,290.0,63.0,111.0,73.0,252.0,371.0,16.0,1.0
3,Ashland City Police Department,Cheatham,0.0,1.0,19.0,0.0,0.0,16.0,0.0,60.0,17.0,30.0,47.0,104.0,283.0,21.0,0.0
4,Kingston Springs Police Department,Cheatham,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,1.0,4.0,2.0,17.0,39.0,2.0,0.0


In [36]:
data.to_feather('../../Outputs/TBI_CrimeType_2021')